![Egeria Logo](https://raw.githubusercontent.com/odpi/egeria/master/assets/img/ODPi_Egeria_Logo_color.png)

### Egeria Hands-On Lab
# Welcome to the Automated Curation Lab

**NOTE - this lab is still under construction and should not be used**

## Introduction

Egeria is an open source project that provides open standards and implementation libraries to connect tools, catalogs and platforms together so they can share information about data and technology (called metadata).

In the [Building a Data Catalog](building-a-data-catalog.ipynb) lab, Peter Profile and Erin Overivew
manually catalogued the weekly measurement files for the Drop Foot clinical trial.

In this hands-on lab you will get a chance to work with Egeria's governance servers to
automate this onboarding process.

## The scenario

[Coco Pharmaceuticals](https://opengovernance.odpi.org/coco-pharmaceuticals/)
is conducting a clinical trial with two hospitals: Oak Dean Hospital and Old Market Hospital.
Each week the two hospitals send Coco Pharmaceuticals a set of measurements from the patients
involved in the trial.  These measurements are located in a CSV file that the hospital sends through
secure file transfer to a folder in Coco Pharmaceutical's landing area.

These files need to be copied into the data lake and catalogued so that they are only visible to the
staff involved in the clinical trial.  It is also important that the lineage of these files is
maintained so the source of the data can be traced.  This process is shown in Figure 1.

![Scenario](../images/automated-curation-scenario.png)
> **Figure 1:** Clinical trial weekly measurements onboarding process

Peter Profile and Erin Overview are responsible for this onboarding process.
![Peter and Erin](../images/peter-and-erin.png)

They have defined a list of requirements for the process:

* Files must be in the landing area for a minimum amount of time.
* As a new file is received, it needs to be catalogued, including:
   * Description
   * Connection details to enable the data scientists to access the contents
   * Column details
   * Governance zones defining the files' visibility
   * Owner 
* A file is not accessible by any of the data lake users until the cataloguing process is complete.
* They must record lineage of each measurements file so they know which hospital it came from.

They have been [manually cataloguing the measurements files](building-a-data-catalog.ipynb) for
the first few weeks to prove the approach but now it is time to automate the process.

They plan to use an
[Integration Daemon](https://egeria.odpi.org/open-metadata-implementation/admin-services/docs/concepts/integration-daemon.html)
called **exchangeDL01** to capture the technical metadata of the files.
Then the 
[Engine Host](https://egeria.odpi.org/open-metadata-implementation/admin-services/docs/concepts/engine-host.html)
server called **governDL01** will manage the move of the file into the data lake,
the augmentation of the metadata properties of the files and the creation of the lineage.

## Setting up

Coco Pharmaceuticals make widespread use of Egeria for tracking and managing their data and related assets.
Figure 1 below shows their servers and the Open Metadata and Governance (OMAG) Server Platforms that are hosting them.

![Figure 1](../images/coco-pharmaceuticals-systems-omag-server-platforms.png)
> **Figure 1:** Coco Pharmaceuticals' OMAG Server Platforms

The code below checks that the platforms are running.  It checks that the servers are configured and then if they are running on the platform.  If a server is configured, but not running, it will start it.

Look for the "Done." message that is displayed after the governance servers have started.

In [ ]:
# Start up the metadata servers and the view server
%run ../common/environment-check.ipynb

print("Start up the Governance Servers")
activatePlatform(dataLakePlatformName, dataLakePlatformURL, [governDL01Name, exchangeDL01Name])

print("Done. ")

----
**Stop here**

Callie Quartile is a data scientist at Coco Pharmaceuticals.  She is responsible for analysing the data that is generated during a clinical trial.

![Callie Quartile](https://raw.githubusercontent.com/odpi/data-governance/master/docs/coco-pharmaceuticals/personas/callie-quartile.png)

Callie's userId is `calliequartile`.

In [ ]:
calliesUserId = "calliequartile"

Clinical trial data is stored in Coco Pharmaceutical's data lake as **data sets**.  A data set is a collection of related data.  For example, a data set may be the names, addresses of all patients in a clinical trail.  Or it may be the measurements from a particular test,
of the notes from a physician as a result of a patient consultation.  There is no upper or lower limit on the
size of a data set.  It just has to contain a useful collection of data.

In this hands-on lab Callie wants to create clusters of patients involved in the trial based on their characteristics.
This will help her look for patterns in the measurement data they receive through the clinical trial.

# Setting up

Coco Pharmaceuticals make widespread use of ODPi Egeria for tracking and managing their data and related assets.
Figure 1 below shows the metadata servers and the platforms that are hosting them.

![Figure 1](../images/coco-pharmaceuticals-systems-omag-server-platforms-metadata-server.png)
> **Figure 1:** Coco Pharmaceuticals' OMAG Server Platforms

The code below checks that the platforms are running.  It checks that the servers are configured and then if they are running on the platform.  If a server is configured, but not running, it will start it.

Look for the "Done." message.  This appears when `environment-check` has finished.

In [ ]:
%run ../common/environment-check.ipynb

----
Callie is using the research team's metadata server called `cocoMDS3`. This server is hosted on the Core OMAG Server Platform.

If any of the platforms are not running, follow [this link to set up and run the platform](https://egeria.odpi.org/open-metadata-resources/open-metadata-labs/).  If any server is reporting that it is not configured then
run the steps in the [Server Configuration](../egeria-server-config.ipynb) lab to configure
the servers.  Then re-run the previous step to ensure all of the servers are started.

----
## Setting up the metadata repository

The code below catalogs an asset in the `cocoMDS1` metadata repository.  It is mimicking the work of the
data lake operations team when they set up a new asset.
Once the set up is complete then Callie can begin to explore the asset's properties.

In [ ]:
displayName = "Drop Foot Clinical Trial Patients"
description = "List of patients registered for the drop foot clinical trial."
fullPath    = "file://secured/research/clinical-trials/drop-foot/Patients.csv"
assetName   = "Patient Asset"

assetGUIDs = assetOwnerCreateCSVAsset(cocoMDS1Name, cocoMDS1PlatformName, cocoMDS1PlatformURL, petersUserId, displayName, description, fullPath)

assetGUID = getLastGUID(assetGUIDs)

print("Created " + assetName)

commentType = "STANDARD_COMMENT"
commentText = "The hospital is still recruiting patients and so we are expecting a new version of this file at the end of week 4"
isPublic    = True

commentGUID = addCommentToAsset(cocoMDS1Name,
                                cocoMDS1PlatformName,
                                cocoMDS1PlatformURL,
                                petersUserId,
                                assetGUID,
                                commentText,
                                commentType,
                                isPublic)

patientAssetOwner = "tanyatidie"
patientOwnerType  = "USER_ID"

addOwner(cocoMDS2Name, cocoMDS2PlatformName, cocoMDS2PlatformURL, erinsUserId, assetName, assetGUID, patientAssetOwner, patientOwnerType)
addZones(cocoMDS2Name, cocoMDS2PlatformName, cocoMDS2PlatformURL, erinsUserId, assetName, assetGUID, ["data-lake", "clinical-trials"])

print("Done.")

----
One you see the message "Done." then the asset is ready for Callie to explore.

---
## Retrieving information about an asset

The call below looks for all assets that contain the string "Patient".  This should include the new
asset that was just created.  The value(s) returned are the unique identifiers for the assets.

In [ ]:
assetConsumerSearchForAssets(cocoMDS3Name, cocoMDS3PlatformName, cocoMDS3PlatformURL, calliesUserId, ".*Patient.*")

----
The search results show that Callie is able to locate the asset. The call below retrieves the core information about the asset along with an indication of what other information is available about the asset.

----

In [ ]:
printAssetUniverse(cocoMDS3Name, cocoMDS3PlatformName, cocoMDS3PlatformURL, calliesUserId, assetGUID)

----